In [2]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [42]:
train = pd.read_csv("output_data.csv", encoding='utf-8')
train

,Unnamed: 0.1,Unnamed: 0,민원 내용,답변 담당부서
0,0,0,불연성 쓰레기봉투 판매하는곳 어디어디 있나요?,경제환경국 자원순환과
1,1,1,1 금정구청 에서 청년의 생활 수준 향상과 생계 주거비 부담 완화를 위해 대학생 생...,경제환경국 일자리경제과
2,2,2,안녕하세요?\n동현로 67 위로 소심유치원과 맞닿은 길이 있습니다. 아침에 3살 아...,안전도시국 교통행정과
3,3,3,구민 을 위해 늘 수고하시는공무원 여러분께 감사합니다 다음과같이민원을제기합니다 금정...,안전도시국 건설과
4,4,4,선두구동 거주민입니다.\n\n집 뒤 물류창고의 새벽 상/하차 작업으로 인한 생활불편...,경제환경국 환경위생과
...,...,...,...,...
3800,3800,4472,경동택배 관련 민원이 저 뿐만 아니라 많이 발생하고 있네요.\n\n앞서 제기한 민원...,안전도시국 교통행정과
3801,3801,4473,안녕하세요. 수고 많으십니다.\n\n1. 저는 금정구 구민으로 온천천을 걷는 운동을...,안전도시국 교통행정과
3802,3802,4474,금정구 오륜동에 현재 가정오수정화시설공사를 하고 있는데 저희집584번지(어머니만거처...,안전도시국 건설과
3803,3803,4475,안녕하세요. 국민을 위한 노고에 수고가 많으십니다.\n\n다름이 아니라 제가 거주하...,경제환경국 환경위생과


In [44]:
test = pd.read_csv('test.csv')
test = pd.DataFrame(test, columns=['text'])

In [45]:
train['답변 담당부서'].value_counts()

답변 담당부서
안전도시국 건설과       757
안전도시국 교통행정과     644
안전도시국 건축과       483
경제환경국 환경위생과     413
경제환경국 공원녹지과     346
안전도시국 안전관리과     342
경제환경국 자원순환과     209
행정지원국 총무과       181
경제환경국 일자리경제과     99
보건소 보건행정과        83
기획감사실            79
행정지원국 민원여권과      59
문화복지국 사회복지과      59
안전도시국 도시관리과      51
Name: count, dtype: int64

In [81]:
train = train[['민원 내용', '답변 담당부서']]

In [82]:
train.shape

(3805, 2)

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   민원 내용    3805 non-null   object
 1   답변 담당부서  3805 non-null   object
dtypes: object(2)
memory usage: 59.6+ KB


In [84]:
from konlpy.tag import Okt

In [85]:
okt = Okt()

# 이랬던 문장이.
print("원본 문장 : ", train.iloc[0]['민원 내용']) 

# 이렇게 바뀝니다. okt.morphs가 형태소 분석기에요.
token_okt = okt.morphs(train.iloc[0]['민원 내용'])
print("형태소 기준으로 tokenize : ", token_okt) 

원본 문장 :  불연성 쓰레기봉투 판매하는곳 어디어디 있나요?
형태소 기준으로 tokenize :  ['불', '연성', '쓰레기봉투', '판매', '하는', '곳', '어디', '어디', '있나요', '?']


In [86]:
# 형태소로 토크나이징
temp = []
for i in train['민원 내용']:
    temp.append(okt.morphs(i))
    
temp = pd.Series(temp)
train['token'] = temp

In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   민원 내용    3805 non-null   object
 1   답변 담당부서  3805 non-null   object
 2   token    3805 non-null   object
dtypes: object(3)
memory usage: 89.3+ KB


In [88]:
from gensim.models import Word2Vec

In [89]:
train

,민원 내용,답변 담당부서,token
0,불연성 쓰레기봉투 판매하는곳 어디어디 있나요?,경제환경국 자원순환과,"[불, 연성, 쓰레기봉투, 판매, 하는, 곳, 어디, 어디, 있나요, ?]"
1,1 금정구청 에서 청년의 생활 수준 향상과 생계 주거비 부담 완화를 위해 대학생 생...,경제환경국 일자리경제과,"[1, 금정, 구청, 에서, 청년, 의, 생활, 수준, 향상, 과, 생계, 주거, ..."
2,안녕하세요?\n동현로 67 위로 소심유치원과 맞닿은 길이 있습니다. 아침에 3살 아...,안전도시국 교통행정과,"[안녕하세요, ?, \n, 동현, 로, 67, 위로, 소심, 유치원, 과, 맞닿은,..."
3,구민 을 위해 늘 수고하시는공무원 여러분께 감사합니다 다음과같이민원을제기합니다 금정...,안전도시국 건설과,"[구민, 을, 위해, 늘, 수고, 하시는, 공무원, 여러분, 께, 감사합니다, 다음..."
4,선두구동 거주민입니다.\n\n집 뒤 물류창고의 새벽 상/하차 작업으로 인한 생활불편...,경제환경국 환경위생과,"[선두구동, 거, 주민, 입니다, ., \n\n, 집, 뒤, 물류, 창고, 의, 새..."
...,...,...,...
3800,경동택배 관련 민원이 저 뿐만 아니라 많이 발생하고 있네요.\n\n앞서 제기한 민원...,안전도시국 교통행정과,"[경, 동, 택배, 관련, 민원, 이, 저, 뿐, 만, 아니라, 많이, 발생, 하고..."
3801,안녕하세요. 수고 많으십니다.\n\n1. 저는 금정구 구민으로 온천천을 걷는 운동을...,안전도시국 교통행정과,"[안녕하세요, ., 수고, 많으십니다, ., \n\n, 1, ., 저, 는, 금정구..."
3802,금정구 오륜동에 현재 가정오수정화시설공사를 하고 있는데 저희집584번지(어머니만거처...,안전도시국 건설과,"[금정구, 오륜동, 에, 현재, 가정, 오, 수정, 화, 시설, 공사, 를, 하고,..."
3803,안녕하세요. 국민을 위한 노고에 수고가 많으십니다.\n\n다름이 아니라 제가 거주하...,경제환경국 환경위생과,"[안녕하세요, ., 국민, 을, 위, 한, 노고, 에, 수고, 가, 많으십니다, ...."


In [90]:
# Word2Vec 모델 학습
wv_model = Word2Vec(train['token'], vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [91]:
import joblib

joblib.dump(wv_model, 'word_model.joblib')

['word_model.joblib']

In [92]:
wv_model.wv.vectors.shape

(10546, 1000)

In [93]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model.wv[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)
    
    return sent_embedding_vectors

In [94]:
train_data_vecs = get_dataset(train['token'], wv_model, 1000)

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
target = '답변 담당부서'
x = train.drop(target, axis=1)
y = train[target]

In [97]:
x

,민원 내용,token
0,불연성 쓰레기봉투 판매하는곳 어디어디 있나요?,"[불, 연성, 쓰레기봉투, 판매, 하는, 곳, 어디, 어디, 있나요, ?]"
1,1 금정구청 에서 청년의 생활 수준 향상과 생계 주거비 부담 완화를 위해 대학생 생...,"[1, 금정, 구청, 에서, 청년, 의, 생활, 수준, 향상, 과, 생계, 주거, ..."
2,안녕하세요?\n동현로 67 위로 소심유치원과 맞닿은 길이 있습니다. 아침에 3살 아...,"[안녕하세요, ?, \n, 동현, 로, 67, 위로, 소심, 유치원, 과, 맞닿은,..."
3,구민 을 위해 늘 수고하시는공무원 여러분께 감사합니다 다음과같이민원을제기합니다 금정...,"[구민, 을, 위해, 늘, 수고, 하시는, 공무원, 여러분, 께, 감사합니다, 다음..."
4,선두구동 거주민입니다.\n\n집 뒤 물류창고의 새벽 상/하차 작업으로 인한 생활불편...,"[선두구동, 거, 주민, 입니다, ., \n\n, 집, 뒤, 물류, 창고, 의, 새..."
...,...,...
3800,경동택배 관련 민원이 저 뿐만 아니라 많이 발생하고 있네요.\n\n앞서 제기한 민원...,"[경, 동, 택배, 관련, 민원, 이, 저, 뿐, 만, 아니라, 많이, 발생, 하고..."
3801,안녕하세요. 수고 많으십니다.\n\n1. 저는 금정구 구민으로 온천천을 걷는 운동을...,"[안녕하세요, ., 수고, 많으십니다, ., \n\n, 1, ., 저, 는, 금정구..."
3802,금정구 오륜동에 현재 가정오수정화시설공사를 하고 있는데 저희집584번지(어머니만거처...,"[금정구, 오륜동, 에, 현재, 가정, 오, 수정, 화, 시설, 공사, 를, 하고,..."
3803,안녕하세요. 국민을 위한 노고에 수고가 많으십니다.\n\n다름이 아니라 제가 거주하...,"[안녕하세요, ., 국민, 을, 위, 한, 노고, 에, 수고, 가, 많으십니다, ...."


In [98]:
x = get_dataset(x['token'], wv_model, 1000)

In [99]:
y.value_counts()

답변 담당부서
안전도시국 건설과       757
안전도시국 교통행정과     644
안전도시국 건축과       483
경제환경국 환경위생과     413
경제환경국 공원녹지과     346
안전도시국 안전관리과     342
경제환경국 자원순환과     209
행정지원국 총무과       181
경제환경국 일자리경제과     99
보건소 보건행정과        83
기획감사실            79
행정지원국 민원여권과      59
문화복지국 사회복지과      59
안전도시국 도시관리과      51
Name: count, dtype: int64

In [100]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15)

In [101]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

model.fit(x_train, y_train)

y_pred = model.predict(x_val)

In [102]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [103]:
accuracy_score(y_val, y_pred)

0.1733800350262697

In [104]:
f1_score(y_val, y_pred, average="macro")

0.05306305376276897

In [105]:
import joblib
joblib.dump(model, 'model.joblib')

['model.joblib']

In [106]:
import pandas as pd

data = pd.read_csv("output_data.csv")

In [110]:
data.head()

,Unnamed: 0.1,Unnamed: 0,민원 내용,답변 담당부서
0,0,0,불연성 쓰레기봉투 판매하는곳 어디어디 있나요?,경제환경국 자원순환과
1,1,1,1 금정구청 에서 청년의 생활 수준 향상과 생계 주거비 부담 완화를 위해 대학생 생...,경제환경국 일자리경제과
2,2,2,안녕하세요?\n동현로 67 위로 소심유치원과 맞닿은 길이 있습니다. 아침에 3살 아...,안전도시국 교통행정과
3,3,3,구민 을 위해 늘 수고하시는공무원 여러분께 감사합니다 다음과같이민원을제기합니다 금정...,안전도시국 건설과
4,4,4,선두구동 거주민입니다.\n\n집 뒤 물류창고의 새벽 상/하차 작업으로 인한 생활불편...,경제환경국 환경위생과


In [111]:
data['답변 담당부서'].value_counts()

답변 담당부서
안전도시국 건설과       757
안전도시국 교통행정과     644
안전도시국 건축과       483
경제환경국 환경위생과     413
경제환경국 공원녹지과     346
안전도시국 안전관리과     342
경제환경국 자원순환과     209
행정지원국 총무과       181
경제환경국 일자리경제과     99
보건소 보건행정과        83
기획감사실            79
행정지원국 민원여권과      59
문화복지국 사회복지과      59
안전도시국 도시관리과      51
Name: count, dtype: int64

In [113]:
data.loc[data['답변 담당부서'] == '안전도시국 건설과','민원 내용'].iloc[0]

'구민 을 위해 늘 수고하시는공무원 여러분께 감사합니다 다음과같이민원을제기합니다 금정구 두구동1568도로부지를 구청에서측량까지했어나 사후관리를아니하여 도로위 농작물 을 심고 안전망까지설치하여 협소한도로를사용하는 학생 .등 .하교. 주민통행에 불편을 느낌니다 조속히 통행권을 확보와 도로개설및 정비를 부탁합니다'

In [122]:
data.loc[data['답변 담당부서'] == '경제환경국 공원녹지과','민원 내용'].iloc[25]

'안녕하세요. 오늘도 노고에 감사드립니다.\n금정구 공공도서관인 금정북뱅크(금정로 75 부산은행 2층)의 외관에 눈에 띄는 한글 간판이 없고 영문 간판 book bank 밖에 설치되어 있지 않아 영어를 잘 모르는 주민이 공공도서관에 접근하기 어렵습니다. 눈에 잘 띄는 한글 간판이나 안내문을 설치해주시면 좋겠습니다.'